In [5]:
# ===============================================
# Tâche 1 - Collecte & Préparation Multi-Source
# ANIP Challenge : Bénin
# ===============================================

import pandas as pd
from pandas_datareader import wb
import requests
from bs4 import BeautifulSoup
import os

# 📂 Créer dossiers si nécessaires
os.makedirs("datasets/raw", exist_ok=True)
os.makedirs("datasets/final", exist_ok=True)

# ================================
# 1. Indicateurs Banque Mondiale
# ================================
start_year = 2000
end_year = 2023

indicators = {
    # Démographiques
    "SP.POP.TOTL": "population",
    "SP.URB.TOTL": "population_urbaine",
    "SP.POP.GROW": "taux_croissance_pop",
    
    # Économiques
    "NY.GDP.MKTP.CD": "gdp_usd",
    "FP.CPI.TOTL": "inflation",
    "SL.UEM.TOTL.ZS": "taux_chomage",
    "NE.EXP.GNFS.CD": "exportations",
    
    # Sociaux
    "SP.DYN.LE00.IN": "esperance_vie",
    "SE.ADT.LITR.ZS": "alphabetisation",
    "SE.PRM.ENRR": "scolarisation_primaire"
}

# Télécharger tous les indicateurs
df = wb.download(indicator=list(indicators.keys()), country="BJ", start=start_year, end=end_year)
df.rename(columns=indicators, inplace=True)

# Reset index et calcul PIB par habitant
df.reset_index(inplace=True)
df["gdp_per_capita"] = df["gdp_usd"] / df["population"]

# Sauvegarde brute
df.to_csv("datasets/raw/benin_multi_raw.csv", index=False)
print("✅ Données brutes multi-source sauvegardées : datasets/raw/benin_multi_raw.csv")

# ================================
# 2. Scraping indicateurs sportifs (ex. Jeux Olympiques)
# ================================
url = "https://fr.wikipedia.org/wiki/B%C3%A9nin_aux_Jeux_olympiques"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Exemple : extraire le tableau des participations
tables = soup.find_all("table", {"class": "wikitable"})
sport_df = pd.DataFrame()

if tables:
    table = tables[0]  # Premier tableau
    sport_df = pd.read_html(str(table))[0]
    sport_df = sport_df.rename(columns={sport_df.columns[0]: "year"})
    sport_df["year"] = pd.to_numeric(sport_df["year"], errors="coerce")
    # Garder uniquement les années 2000-2023
    sport_df = sport_df[(sport_df["year"] >= start_year) & (sport_df["year"] <= end_year)]
    sport_df = sport_df.sort_values("year").reset_index(drop=True)
    # Exemple colonne : nombre de participants
    if "Participants" in sport_df.columns:
        sport_df.rename(columns={"Participants": "nb_participants_olympiques"}, inplace=True)
else:
    print("⚠️ Aucun tableau sportif trouvé sur la page Wikipédia.")

# Merge des données sportives
if not sport_df.empty:
    df = df.merge(sport_df[["year", "nb_participants_olympiques"]], on="year", how="left")
else:
    df["nb_participants_olympiques"] = None

# ================================
# 3. Nettoyage et vérification
# ================================
df = df.sort_values("year").reset_index(drop=True)
df.dropna(subset=["population", "gdp_usd"], inplace=True)

# Sauvegarde finale
df.to_csv("datasets/final/benin_multi.csv", index=False)
print("✅ Dataset final multi-source sauvegardé : datasets/final/benin_multi.csv")

# ================================
# 4. Glossaire des variables
# ================================
glossaire = pd.DataFrame({
    "Variable": [
        "year", "population", "population_urbaine", "taux_croissance_pop", 
        "gdp_usd", "gdp_per_capita", "inflation", "taux_chomage", "exportations",
        "esperance_vie", "alphabetisation", "scolarisation_primaire", "nb_participants_olympiques"
    ],
    "Nom complet": [
        "Année", "Population totale", "Population urbaine", "Taux de croissance de la population",
        "PIB", "PIB par habitant", "Inflation", "Taux de chômage", "Exportations",
        "Espérance de vie", "Taux d'alphabétisation adulte", "Taux de scolarisation primaire", "Participants aux JO"
    ],
    "Définition": [
        "Année de la mesure",
        "Nombre total d’habitants",
        "Nombre d’habitants en milieu urbain",
        "Variation annuelle de la population totale",
        "Produit Intérieur Brut en USD courants",
        "PIB par habitant = PIB / population",
        "Variation annuelle des prix à la consommation",
        "Proportion de la population active au chômage",
        "Valeur totale des exportations de biens et services",
        "Espérance de vie à la naissance",
        "Pourcentage d’adultes alphabétisés",
        "Taux de scolarisation dans l’enseignement primaire",
        "Nombre de participants béninois aux Jeux Olympiques"
    ],
    "Unité": [
        "année", "personnes", "personnes", "%", "USD", "USD/personne", "%", "%", "USD",
        "années", "%", "%", "personnes"
    ],
    "Source": [
        "Banque Mondiale"]*12 + ["Wikipedia"],
    "Période": ["2000-2023"]*13,
    "Géographie": ["Bénin"]*13
})

glossaire.to_csv("datasets/final/glossaire_variables.csv", index=False)
print("✅ Glossaire sauvegardé : datasets/final/glossaire_variables.csv")

# ================================
# 5. Fichier anomalies
# ================================
anomalies = df[(df["population"] <= 0) | (df["gdp_usd"] <= 0) | (df["gdp_per_capita"] <= 0)]
anomalies.to_csv("datasets/final/anomalies.csv", index=False)
print("✅ Fichier anomalies sauvegardé : datasets/final/anomalies.csv")


/tmp/ipykernel_11676/2218158304.py:41: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = wb.download(indicator=list(indicators.keys()), country="BJ", start=start_year, end=end_year)


✅ Données brutes multi-source sauvegardées : datasets/raw/benin_multi_raw.csv
⚠️ Aucun tableau sportif trouvé sur la page Wikipédia.
✅ Dataset final multi-source sauvegardé : datasets/final/benin_multi.csv
✅ Glossaire sauvegardé : datasets/final/glossaire_variables.csv
✅ Fichier anomalies sauvegardé : datasets/final/anomalies.csv
